In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import IPython

In [52]:
#!pip install -q -U keras-tuner
import kerastuner as kt

ImportError: cannot import name 'preprocessing' from 'tensorflow.keras.layers.experimental' (C:\Users\Neil\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_core\python\keras\api\_v1\keras\layers\experimental\__init__.py)

In [39]:
print("NumPy version " + np.__version__)
print("Tensorflow version " + tf.__version__)

NumPy version 1.18.5
Tensorflow version 1.15.4


In [40]:
## Data Reading ##
#tf.data.experimental.CsvDataset("data2/train_drug.csv", "float32")
train_features = pd.read_csv("data/train_features.csv")
results = pd.read_csv("data/train_targets_scored.csv")
testSet = pd.read_csv("data/test_features.csv")

In [41]:
## Parameters ##
validationProportion = 0.2

N = len(train_features) #Nombre d'échantillons testés
I = train_features.shape[1]-1 #Nombre de input
M = results.shape[1]-1 #Nombre de pathologies obserables

In [42]:
## HYPER parameters ##
nbEpoch = 30 
learningRate = 0.01 #with the adam optimizer

# nombre de couches cachées
# nombre de neurones
# fonctions d'activation
# epsilon (adam)

# cf randomizedSearchCV

In [43]:
## Normalization ##
maxVal=np.max(np.max(np.abs(train_features.iloc[:,4:])))
train_features.iloc[:,4:] = train_features.iloc[:,4:]/maxVal
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,0.10620,0.05577,-0.02479,-0.06208,-0.01944,-0.10120,...,0.02862,0.02584,0.08076,0.05523,-0.01912,0.06584,-0.03981,0.02139,0.03801,0.04176
1,id_000779bfc,trt_cp,72,D1,0.00743,0.04087,0.02991,0.00604,0.10190,0.05207,...,-0.04265,0.07543,0.04708,0.00230,0.02957,0.04899,0.01522,0.01241,0.06077,0.07371
2,id_000a6266a,trt_cp,48,D1,0.06280,0.05817,0.15540,-0.00764,-0.00323,0.12390,...,-0.07250,-0.06297,0.06103,0.00223,-0.13240,-0.03174,-0.06417,-0.02187,-0.14080,0.06931
3,id_0015fd391,trt_cp,48,D1,-0.05138,-0.02491,-0.02656,0.05288,0.40620,-0.08095,...,-0.20990,-0.06441,-0.56300,-0.13780,-0.08632,-0.12880,-0.16210,-0.08784,-0.03876,-0.08154
4,id_001626bd3,trt_cp,72,D2,-0.03254,-0.04009,0.09700,0.06919,0.14180,-0.08244,...,0.00042,0.00048,0.06670,0.10690,0.05523,-0.03031,0.01094,0.02885,-0.03786,0.07125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.01394,-0.00636,-0.01112,-0.05080,-0.04713,0.07201,...,0.01969,0.00262,-0.08121,0.03434,0.05372,-0.03246,0.00631,0.09171,0.05258,0.04680
23810,id_fffb70c0c,trt_cp,24,D2,-0.13260,0.03478,-0.03743,0.09905,-0.07178,0.06621,...,0.04286,0.04426,0.00423,-0.03195,-0.08086,-0.09798,-0.02084,-0.01224,-0.02715,0.03689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.03942,0.03756,0.03109,-0.07389,0.05505,-0.00159,...,0.05409,0.03755,0.07343,0.02807,0.04116,0.06422,0.02256,0.07592,0.06656,0.03808
23812,id_fffcb9e7c,trt_cp,24,D1,0.06660,0.02324,0.04392,0.02044,0.08531,-0.00343,...,-0.01105,0.04258,-0.02012,0.01506,0.15230,0.07101,0.01732,0.07015,-0.06290,0.00740


In [44]:
## replacing sig_id by drug_id ##
dataSet = train_features.join(results, lsuffix='sig_id', rsuffix='sig_id')
dataSet = dataSet.drop(columns=['sig_idsig_id'])

dataSet

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,trt_cp,24,D1,0.10620,0.05577,-0.02479,-0.06208,-0.01944,-0.10120,-0.10220,...,0,0,0,0,0,0,0,0,0,0
1,trt_cp,72,D1,0.00743,0.04087,0.02991,0.00604,0.10190,0.05207,0.02341,...,0,0,0,0,0,0,0,0,0,0
2,trt_cp,48,D1,0.06280,0.05817,0.15540,-0.00764,-0.00323,0.12390,0.01715,...,0,0,0,0,0,0,0,0,0,0
3,trt_cp,48,D1,-0.05138,-0.02491,-0.02656,0.05288,0.40620,-0.08095,-0.19590,...,0,0,0,0,0,0,0,0,0,0
4,trt_cp,72,D2,-0.03254,-0.04009,0.09700,0.06919,0.14180,-0.08244,-0.02800,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,trt_cp,24,D2,0.01394,-0.00636,-0.01112,-0.05080,-0.04713,0.07201,0.05773,...,0,0,0,0,0,0,0,0,0,0
23810,trt_cp,24,D2,-0.13260,0.03478,-0.03743,0.09905,-0.07178,0.06621,-0.02252,...,0,0,0,0,0,0,0,0,0,0
23811,ctl_vehicle,48,D2,0.03942,0.03756,0.03109,-0.07389,0.05505,-0.00159,-0.02541,...,0,0,0,0,0,0,0,0,0,0
23812,trt_cp,24,D1,0.06660,0.02324,0.04392,0.02044,0.08531,-0.00343,0.00323,...,0,0,0,0,0,0,0,0,0,0


In [45]:
## Shuffling ##
dataSet = dataSet.reindex(np.random.permutation(dataSet.index))
dataSet

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
10823,trt_cp,72,D1,-0.01765,0.05094,0.09475,-0.12370,-0.01196,0.01165,-0.02010,...,0,0,0,0,0,0,0,0,0,0
11738,trt_cp,72,D2,-0.01012,0.02095,-0.04502,0.03743,0.07837,-0.13670,-0.07312,...,0,0,0,0,0,0,0,0,0,0
19915,trt_cp,48,D1,-0.11210,0.02386,0.10990,0.03000,-0.13260,-0.05957,-0.08717,...,0,0,0,0,0,0,0,0,0,0
2942,trt_cp,72,D1,-0.03708,0.10300,-0.01507,-0.00617,-0.03163,-0.02854,-0.00360,...,0,0,0,0,0,0,0,0,0,0
19648,trt_cp,72,D1,0.21960,-0.00063,0.11400,0.06506,-0.11250,-0.25050,0.05560,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,trt_cp,24,D1,-0.12850,0.03773,-0.04140,0.04165,0.07517,0.23190,0.06144,...,0,0,0,0,0,0,0,0,0,0
20288,ctl_vehicle,48,D1,-0.01555,-0.09171,0.15390,-0.10390,0.20110,-0.00001,-0.08884,...,0,0,0,0,0,0,0,0,0,0
14108,trt_cp,24,D2,-0.00200,-0.04248,0.05984,-0.01459,-0.02566,0.04059,-0.07975,...,0,0,0,0,0,0,0,0,0,0
14158,trt_cp,48,D2,-0.01322,0.02298,-0.11750,-0.15990,0.01595,0.05821,0.00055,...,0,0,0,0,0,0,0,0,0,0


In [46]:
## Replacing categories values by numbers ##
features_to_convert=['cp_type','cp_time','cp_dose']
for feat in features_to_convert:
    dataSet[feat] = pd.Categorical(dataSet[feat])
    dataSet[feat] = dataSet[feat].cat.codes
    
dataSet

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
10823,1,2,0,-0.01765,0.05094,0.09475,-0.12370,-0.01196,0.01165,-0.02010,...,0,0,0,0,0,0,0,0,0,0
11738,1,2,1,-0.01012,0.02095,-0.04502,0.03743,0.07837,-0.13670,-0.07312,...,0,0,0,0,0,0,0,0,0,0
19915,1,1,0,-0.11210,0.02386,0.10990,0.03000,-0.13260,-0.05957,-0.08717,...,0,0,0,0,0,0,0,0,0,0
2942,1,2,0,-0.03708,0.10300,-0.01507,-0.00617,-0.03163,-0.02854,-0.00360,...,0,0,0,0,0,0,0,0,0,0
19648,1,2,0,0.21960,-0.00063,0.11400,0.06506,-0.11250,-0.25050,0.05560,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,1,0,0,-0.12850,0.03773,-0.04140,0.04165,0.07517,0.23190,0.06144,...,0,0,0,0,0,0,0,0,0,0
20288,0,1,0,-0.01555,-0.09171,0.15390,-0.10390,0.20110,-0.00001,-0.08884,...,0,0,0,0,0,0,0,0,0,0
14108,1,0,1,-0.00200,-0.04248,0.05984,-0.01459,-0.02566,0.04059,-0.07975,...,0,0,0,0,0,0,0,0,0,0
14158,1,1,1,-0.01322,0.02298,-0.11750,-0.15990,0.01595,0.05821,0.00055,...,0,0,0,0,0,0,0,0,0,0


In [47]:
## Splitting into training and validation ##
trainingSize = int((1-validationProportion) * N)

trainingSet = dataSet.iloc[:trainingSize,:] 
validationSet = dataSet.iloc[trainingSize:,:]

print("Training size:", len(trainingSet))
print("Validation size:", len(validationSet))

Training size: 19051
Validation size: 4763


In [48]:
XTrain = trainingSet.iloc[:,:I]
YTrain = trainingSet.iloc[:,I:]
XTrain

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
10823,1,2,0,-0.01765,0.05094,0.09475,-0.12370,-0.01196,0.01165,-0.02010,...,-0.02271,-0.14520,-0.13090,0.02251,-0.04011,-0.06198,0.05065,-0.01110,-0.04415,-0.04101
11738,1,2,1,-0.01012,0.02095,-0.04502,0.03743,0.07837,-0.13670,-0.07312,...,-0.00714,0.08559,0.08107,-0.01021,0.02237,-0.07071,0.07065,-0.02180,0.02334,-0.00589
19915,1,1,0,-0.11210,0.02386,0.10990,0.03000,-0.13260,-0.05957,-0.08717,...,0.07039,0.04574,0.10420,0.08200,-0.16110,0.07931,0.02147,0.00862,0.08983,-0.03522
2942,1,2,0,-0.03708,0.10300,-0.01507,-0.00617,-0.03163,-0.02854,-0.00360,...,0.03966,0.02653,0.11220,0.03942,0.01882,-0.00529,0.05715,0.02337,-0.02340,0.04591
19648,1,2,0,0.21960,-0.00063,0.11400,0.06506,-0.11250,-0.25050,0.05560,...,-0.17990,-0.21930,-0.33760,0.06643,-0.20970,0.04423,-0.63160,-0.14410,-0.35380,-0.14110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15607,1,2,1,-0.09154,-0.06904,-0.00570,0.00430,-0.06234,0.03350,-0.03293,...,0.03013,0.03373,-0.03892,-0.02581,-0.04512,0.02332,0.04231,0.06990,0.06840,-0.04747
6816,1,1,0,0.03519,0.03332,0.14810,-0.00236,-0.02471,0.12010,-0.08708,...,-0.09203,0.00582,-0.06117,-0.02314,-0.03344,-0.07517,0.00474,-0.05414,0.01101,-0.06033
21450,1,2,0,0.01063,0.07304,-0.04408,-0.08997,-0.03692,-0.01025,-0.05217,...,-0.20650,-0.12100,-0.06116,-0.05210,-0.08846,-0.00174,-0.06964,-0.11760,-0.08495,0.02549
4818,1,1,0,-0.03061,0.06330,-0.00047,0.02095,-0.03231,-0.03286,-0.09164,...,0.04858,0.04254,0.00144,0.02674,-0.01204,0.03833,-0.16350,-0.03438,0.05605,-0.08851


In [49]:
## Batching ##
"""
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

trainingds = df_to_dataset(trainingSet)
trainingds
"""

'\n# A utility method to create a tf.data dataset from a Pandas Dataframe\ndef df_to_dataset(dataframe, shuffle=True, batch_size=32):\n    dataframe = dataframe.copy()\n    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), dataframe))\n    if shuffle:\n        ds = ds.shuffle(buffer_size=len(dataframe))\n    ds = ds.batch(batch_size)\n    ds = ds.prefetch(batch_size)\n    return ds\n\ntrainingds = df_to_dataset(trainingSet)\ntrainingds\n'

In [50]:
## Training ##
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(M, activation=tf.nn.sigmoid)])

#sigmoid is more appropriate rather softmax because we are multiple factors of cancer 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
model.fit(XTrain, YTrain, epochs=nbEpoch)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=       cp_type  cp_time  cp_dose      g-0      g-1      g-2      g-3      g-4  \
10823        1        2        0 -0.01765  0.05094  0.09475 -0.12370 -0.01196   
11738        1        2        1 -0.01012  0.02095 -0.04502  0.03743  0.07837   
19915        1        1        0 -0.11210  0.02386  0.10990  0.03000 -0.13260   
2942         1        2        0 -0.03708  0.10300 -0.01507 -0.00617 -0.03163   
19648        1        2        0  0.21960 -0.00063  0.11400  0.06506 -0.11250   
...        ...      ...      ...      ...      ...      ...      ...      ...   
15607        1        2        1 -0.09154 -0.06904 -0.00570  0.00430 -0.06234   
6816         1        1        0  0.03519  0.03332  0.14810 -0.00236 -0.02471   
21450        1        2        0  0.01063  0.07304 -0.04408 -0.08997 -0.03692   
4818         1        1        0 -0.03061  0.06330 -0.00047  0.02095 -0.03231   
1165

In [ ]:
#Perform Hyperparameters
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(XTrain, YTrain, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# We relaunch the system with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))